# Question and Answer with Babi
reference: https://github.com/keras-team/keras/blob/master/examples/babi_memnn.py

In [2]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from functools import reduce
import tarfile
import numpy as np
import re

Using TensorFlow backend.


In [3]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]

def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true, only the sentences
    that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data


def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file,
    retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data

In [6]:
challenge = 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt'
with tarfile.open('babi_tasks_1-20_v1-2.tar.gz') as tar:
    train_stories = get_stories(tar.extractfile(challenge.format('train')))
    test_stories = get_stories(tar.extractfile(challenge.format('test')))

/home/jupyter/.virtualenvs/gym/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [22]:
story, question, answer = train_stories[10]
print(story)
print(question)
print(answer)

['Mary', 'went', 'to', 'the', 'bedroom', '.', 'John', 'journeyed', 'to', 'the', 'bathroom', '.']
['Where', 'is', 'John', '?']
bathroom


In [7]:
vocab = set()
for story, q, answer in train_stories + test_stories:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)
word2idx = dict((c, i + 1) for i, c in enumerate(vocab))

In [8]:
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

In [56]:
print(vocab_size)

22


In [11]:
def vectorize_stories(data, word2idx=word2idx, story_maxlen=story_maxlen, query_maxlen=query_maxlen):
    inputs, queries, answers = [], [], []
    for story, query, answer in data:
        inputs.append([word2idx[w] for w in story])
        queries.append([word2idx[w] for w in query])
        answers.append(word2idx[answer])
    return (pad_sequences(inputs, maxlen=story_maxlen),
            pad_sequences(queries, maxlen=query_maxlen),
            np.array(answers))

In [12]:
inputs_train, queries_train, answers_train = vectorize_stories(train_stories)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories)

In [14]:
print(inputs_train.shape, queries_train.shape, answers_train.shape)
print(inputs_test.shape, queries_test.shape, answers_test.shape)

(10000, 68) (10000, 4) (10000,)
(1000, 68) (1000, 4) (1000,)


In [23]:
print(inputs_train[10])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  5 21 19 18 10  1  4 14 19 18  9  1]


# memory network model

In [25]:
story_encoder_m = Sequential()
story_encoder_m.add(Embedding(input_dim=vocab_size, output_dim=64))
story_encoder_m.add(Dropout(0.2))

story_encoder_c = Sequential()
story_encoder_c.add(Embedding(input_dim=vocab_size, output_dim=query_maxlen))
story_encoder_c.add(Dropout(0.2))

In [26]:
story_sequence = Input((story_maxlen,))
story_encoded_m = input_encoder_m(input_sequence)
story_encoded_c = input_encoder_c(input_sequence)

In [27]:
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size, output_dim=64, input_length=query_maxlen))
question_encoder.add(Dropout(0.2))

In [28]:
question = Input((query_maxlen,))
question_encoded = question_encoder(question)

In [29]:
match = dot([story_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)
response = add([match, story_encoded_c]) 
response = Permute((2, 1))(response)

In [30]:
answer = concatenate([response, question_encoded])
answer = LSTM(32)(answer) 

answer = Dropout(0.2)(answer)
answer = Dense(vocab_size)(answer) 
answer = Activation('softmax')(answer)

In [31]:
model = Model([story_sequence, question], answer)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# train
model.fit([inputs_train, queries_train], answers_train,
          batch_size=32,
          epochs=120,
          validation_data=([inputs_test, queries_test], answers_test))

Train on 10000 samples, validate on 1000 samples
Epoch 1/120
10000/10000 [==============================] - 3s 268us/step - loss: 1.9762 - acc: 0.1657 - val_loss: 1.8036 - val_acc: 0.1540
Epoch 2/120
10000/10000 [==============================] - 2s 199us/step - loss: 1.7690 - acc: 0.2215 - val_loss: 1.6227 - val_acc: 0.3590
Epoch 3/120
10000/10000 [==============================] - 2s 200us/step - loss: 1.5582 - acc: 0.3623 - val_loss: 1.4839 - val_acc: 0.4030
Epoch 4/120
10000/10000 [==============================] - 2s 200us/step - loss: 1.4643 - acc: 0.4249 - val_loss: 1.4112 - val_acc: 0.4200
Epoch 5/120
10000/10000 [==============================] - 2s 201us/step - loss: 1.3978 - acc: 0.4546 - val_loss: 1.3586 - val_acc: 0.4660
Epoch 6/120
10000/10000 [==============================] - 2s 198us/step - loss: 1.3681 - acc: 0.4670 - val_loss: 1.3529 - val_acc: 0.4630
Epoch 7/120
10000/10000 [==============================] - 2s 202us/step - loss: 1.3444 - acc: 0.4829 - val_loss: 1.3

# Predict

In [65]:
idx = 10
story, question, answer = test_stories[idx]
print(story)
print(question)
print('Correct answer: ', answer)
story_array = np.expand_dims(inputs_test[idx], axis=0)
question_array = np.expand_dims(queries_test[idx], axis=0)
prediction_idx = np.argmax(model.predict([story_array, question_array]))
prediction = vocab[prediction_idx-1]
print('prediction: ' , prediction)

['John', 'travelled', 'to', 'the', 'office', '.', 'Mary', 'journeyed', 'to', 'the', 'kitchen', '.']
['Where', 'is', 'Mary', '?']
Correct answer:  kitchen
prediction:  kitchen


In [66]:
idx = 100
story, question, answer = test_stories[idx]
print(story)
print(question)
print('Correct answer: ', answer)
story_array = np.expand_dims(inputs_test[idx], axis=0)
question_array = np.expand_dims(queries_test[idx], axis=0)
prediction_idx = np.argmax(model.predict([story_array, question_array]))
prediction = vocab[prediction_idx-1]
print('prediction: ' , prediction)

['Mary', 'travelled', 'to', 'the', 'garden', '.', 'Mary', 'moved', 'to', 'the', 'bedroom', '.']
['Where', 'is', 'Mary', '?']
Correct answer:  bedroom
prediction:  bedroom
